In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
from numpy import linalg as LA
import itertools
np.random.seed(5)#4


In [2]:


# ======================================================================================================================

# Auxiliary functions generating data for the RCLM

# ======================================================================================================================

import numpy as np
import pandas as pd

def get_ccp(u_choice, T, list_J, N):
    ccp = np.zeros((sum(list_J), N))
    for t in range(T):
        u_choice_t = u_choice[sum(list_J[:t]): sum(list_J[:t + 1]), :]
        ccp[sum(list_J[:t]): sum(list_J[:t + 1]), :] = np.exp(u_choice_t) / (1 + np.exp(u_choice_t).sum(0).reshape(1, N))
    return ccp

def get_shares(lam, v, x_1, delta, T, J, N):
    # Calculate individual's utility
    list_J = T * [J]
    K_rc = x_1.shape[1]
    u_choice = np.zeros((sum(list_J), N)) + np.repeat(delta, N, axis=1)
    for i in range(K_rc):
        u_choice += x_1[:, i: i + 1] * v[:, i * N: (i + 1) * N]*lam[i]

    # Get CCPs
    ccp = get_ccp(u_choice, T, list_J, N)

    # Calculate market shares
    sigma = np.hstack([ccp[sum(list_J[:t]): sum(list_J[:t + 1]), :].mean(1) for t in range(T)])
    sigma_0 = np.hstack([1 - sum(sigma[sum(list_J[:t]): sum(list_J[:t + 1])]) for t in range(T)])
    sigma_0 = np.hstack([sigma_0[t: t + 1].repeat(list_J[t]) for t in range(T)])
    return sigma, sigma_0

""" Generate Data """
def gen_data(seed, spec):

    # DGP parameters
    
    T = spec['T']
    J = spec['J']
    K_1 = spec['K_1']
    N = spec['N']
    beta_true = spec['beta_true']


    # set the seed
    np.random.seed(seed)

    # generate market indices
    list_J = T * [J]
    ids_market = np.hstack([[t] * list_J[t] for t in range(len(list_J))])

    '''-------------------------------------------------------------------------'''
    ''' 1. Generate raw characteristics and other variables						'''
    '''-------------------------------------------------------------------------'''

    # Non-truncated, normalized design

    x_1 = np.random.normal(loc=0, scale=1, size=(sum(list_J), K_1))
    
    xi = np.random.normal(loc=0, scale=0.15, size=(sum(list_J), 1))
    
    delta = (x_1 @ beta_true)[:, np.newaxis] + xi

    '''-------------------------------------------------------------------------'''
    ''' 2. True features								                        '''
    '''-------------------------------------------------------------------------'''

    
    K_rc = x_1.shape[1]
    v = np.zeros((T, K_rc * N))

    '''-------------------------------------------------------------------------'''
    ''' 3. RC distribution								                        '''
    '''-------------------------------------------------------------------------'''

    # Generate random coefficients
    
    v_t = np.random.normal(loc=0, scale=1, size=(N, K_rc)).reshape(1, K_rc * N)
    for t in range(T):
        v[t, :] = v_t
        # draw different consumers across markets
        # v[t, :] = (np.random.normal(loc=0, scale=1, size=(N, K_rc)) @ cholesky(covx)).T.reshape(1, K_rc * N)
    v = np.vstack([v[t:t+1, :].repeat(list_J[t], axis=0) for t in range(T)])

    '''-------------------------------------------------------------------------'''
    ''' 4. Calculate market shares						                        '''
    '''-------------------------------------------------------------------------'''
    print('Calculating market shares', delta.shape)
    # Calculate market shares
    sigma, sigma_0 = get_shares([1, 1], v, x_1, delta, T, J, N)

    return {'market_ids': ids_market,  'x_1': x_1, 
             's': sigma[:, np.newaxis], 's_0': sigma_0[:, np.newaxis], 'xi': xi, 'v_t':v_t, 'v':v, 'delta':delta}


def get_mat_diff_t(x):
    """
    create characteristic differences for one market
    """
    (J, K) = x.shape
    mat_diff_t = np.empty((0, K*(J-1)))
    for j in range(J):
        #mat_diff_t[current : current + J, :] = x[j, :] - np.delete(x, j, axis=0)
        mat_diff_t = np.r_[mat_diff_t,(np.delete(x, j, axis=0) - x[j, :]).reshape(1,-1)]
    #mat_diff_t = np.tile(x.T, J).T - np.repeat(x, J, axis=0)
    return mat_diff_t


def get_diff(ids_market, x, s=None):
    K = x.shape[1]
    mat_diff = []
    for t in range(min(ids_market), max(ids_market) + 1):
        J_t = int(sum(ids_market == t))
        #mat_diff_t = mat_diff_t.reshape(J_t, (J_t-1) * K)
        if s is not None:
            s_t = s[ids_market == t, :]
            x_t = x[ids_market == t, :]
            s_t_mat = np.empty((0, (J-1)))
            mat_diff_t = np.empty((0, (J-1)))
            omega_t    = []
            for j in range(J_t):
              mat_diff_t = (np.delete(x_t, j, axis=0) - x_t[j, :]).reshape(1,-1).reshape(-1,K)
              s_t_mat    = np.delete(s_t.T, j, axis=1).reshape(1,-1)
              A          = [s_t[j][0]] + list(map(lambda x : tuple(x), np.c_[s_t_mat.T,mat_diff_t]))  
              mat_diff.append(A)
    return pd.DataFrame(mat_diff)

def get_X(ids_market, x):
    K = x.shape[1]
    mat_diff = []
    for t in range(min(ids_market), max(ids_market) + 1):
        J_t = int(sum(ids_market == t))
        x_t = x[ids_market == t, :]
        for j in range(J_t):
                mat_diff_t = (np.delete(x_t, j, axis=0)).reshape(1,-1).reshape(-1,K)
                mat_diff.append([tuple(x_t[j, :])] + (list(map(lambda x : tuple(x), mat_diff_t))))
    return pd.DataFrame(mat_diff)


def get_instruments(ids_market, x):
    K = x.shape[1]
    print(K)
    mat_diff = []
    for t in range(min(ids_market), max(ids_market) + 1):
        J_t = int(sum(ids_market == t))
        #mat_diff_t = mat_diff_t.reshape(J_t, (J_t-1) * K)
        print(J_t)
        x_t = x[ids_market == t, :]
        mat_diff_t = np.empty((0, (J-1)))
        omega_t    = []
        for j in range(J):
          mat_diff_t = (np.delete(x_t, j, axis=0) - x_t[j, :]).reshape(1,-1).reshape(-1,K)
          A          =  list(map(lambda x : tuple(x), mat_diff_t))  
          mat_diff.append(A)
    return pd.DataFrame(mat_diff)




def data_transform(data):

    ### 1. stransform data ###
    # extract the data
    x_1 = data['x_1']
    s = data['s']
    s_0 = data['s_0']
    v   = data['v']
    market_ids = data['market_ids']
    xi = data['xi']
    delta = data['delta']
    
    X = get_X(market_ids, x_1)
    print('Dimension of X vector: %s' % X.shape[1])
    Z = X#get_instruments(market_ids, np.c_[x_1, x_2])
    print('Dimension of Z vector: %s' % Z.shape[1])



    return {'X': X, 'Z': Z, 's': s, 's_0': s_0, 'xi': xi, 'market_ids': market_ids, 
            'v': v, 'x_1': x_1, 'delta': delta}


# Estimate BLP

In [3]:
def compute_delta(lam, v, x_1, delta_0, T, J, N, shares, tol=1e-6):
    err = 1
    while err > tol:
        sigma, sigma_0 = get_shares(lam, v, x_1, delta_0, T, J, N)
        shares = shares.reshape(-1,1)
        sigma = sigma.reshape(-1,1)
        delta_new = shares - sigma + delta_0
        #print(delta_new.shape)
        err = np.max(np.abs(delta_new - delta_0))
        delta_0 = delta_new.copy()
    return delta_0    

In [4]:
def symmetric_inversion(X):
    c = np.linalg.inv(np.linalg.cholesky(X))
    inverse = np.dot(c.T, c)
    return inverse

In [5]:
# verticall concatenate the delta_diffs
from joblib import Parallel, delayed
diff  = 0.0001

def compute_diff(i, theta_2, v, x_1, log_s_s0, T, J, N, s, delta, diff):
    theta_tmp = theta_2.copy() 
    theta_tmp[i] += diff
    delta_tmp = compute_delta(theta_tmp, v, x_1, log_s_s0 , T, J, N, s)
    return (delta_tmp - delta)/diff



In [6]:
def blp_instruments(mat, K_1):
    # Ensure K divides the number of columns
    compet_mat = mat[:,K_1:]
    blp_inst = []
    for i in range(K_1):
        blp_inst.append(np.sum(compet_mat[:,i::K_1], axis=1, keepdims=True))
    blp_inst = np.concatenate(blp_inst, axis=1)
    return np.c_[mat[:,:K_1], blp_inst]

def local_iv_instruments(mat, K_1):
    # Ensure K divides the number of columns
    compet_mat = mat[:,K_1:]
    local_ivs = []
    for i in range(K_1):
        focal_char = mat[:, i:i+1]
        competing_chars = compet_mat[:,i::K_1]
        squared_diff = np.sum((competing_chars - focal_char)**2, axis=1, keepdims=True)
        local_ivs.append(squared_diff)
    local_ivs = np.concatenate(local_ivs, axis=1)
    return np.c_[mat[:,:K_1], local_ivs]


In [7]:
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import torch.nn.functional as F

class MLIV(nn.Module):
    def __init__(self, J, T,  K_1):
        super(MLIV, self).__init__()
        self.H = nn.Sequential(
            nn.Linear(in_features=J*K_1, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=K_1),
        )

        self.J = J
        self.K_1 = K_1
        self.T = T
    
    def get_mliv(self, _X, _Z):
        _HZ  = self.H(_Z)
        _HZC = torch.cat([_X, _HZ], dim=1)
        return _HZC

    
    def forward(self, _X, _Z, _jacobian, _Xi):
        print(_Z.shape)
        _HZ  = self.H(_Z)
        
        _HZC = torch.cat([_X, _HZ], dim=1)
        self.HZC = _HZC
        W        = torch.linalg.inv(_HZC.T @ _HZC)
        moment_grad = torch.mean(_jacobian.unsqueeze(2) * _HZC.unsqueeze(1), dim=0)
        G = -1 * moment_grad
        
        S_0 = (_Xi * _HZC).T @ (_Xi * _HZC) / (self.T * self.J)
        V = torch.linalg.inv(G @ W @ G.T) @ (G @ W @ S_0 @ W @ G.T) @ torch.linalg.inv(G @ W @ G.T) / (self.T * self.J)

        print(torch.diag(V))
        cost = torch.sum(torch.diag(V)[self.K_1:])
        return cost





c:\Users\amdeep\Anaconda3\envs\bond37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def generate_instruments(seed, T = 100, J = 2, K_1 = 2, N   = 2000):
    
    data_spec = { 'T': T, 'J': J,
                'N': N, 'beta_true': np.array([1,  1]), 'K_1': K_1 }
    data_raw = gen_data(seed, data_spec)
    data = data_transform(data_raw)
    x_1  = data['x_1']
    s    = data['s']
    s_0  = data['s_0']
    v    = data['v']
    X   = data['X']
    Z   = data['Z']
    delta_orig = data['delta']
    Z_list = []
    for col in Z.columns:
        Z_list.append(np.vstack(np.array(Z[col].apply(lambda x: np.array(x)))))
    Z_raw = np.concatenate(Z_list, axis=1)
    Z = blp_instruments(Z_raw,  K_1)
    W  = symmetric_inversion(Z.T@Z)
    theta_p = np.ones(2*K_1)/2
    done = False
    iterations = 0
    while not done :
        iterations += 1
        if iterations > 20:
            return None
        theta_1 = theta_p[:K_1]
        theta_2 = theta_p[K_1:]
        delta = compute_delta(theta_2, v, x_1, np.log(s/s_0) , T, J, N, s)
        diff  = 0.0001
        delta_diffs = Parallel(n_jobs=-1)(delayed(compute_diff)(i, theta_2, v, x_1, np.log(s/s_0), T, J, N, s, delta, diff) for i in range(K_1))
        delta_deriv = np.concatenate(delta_diffs, axis=1)    
        xi = delta - (x_1@theta_1).reshape(-1,1)
        g  = np.mean(xi*Z,  axis = 0) 
        jacobian = np.concatenate([-1*x_1,delta_deriv],axis=1)
        moment_grad = np.mean((jacobian[:, :, np.newaxis] * Z[:, np.newaxis, :]), axis = 0)
        G = -1*moment_grad
        #if theta_p[2] > -2:
        #    theta_p[2] = -2.41
        theta_n = theta_p + symmetric_inversion(G @ W @ (G.T)) @ G @ W @ np.reshape(g,[-1,1]).flatten()
        for ih in range(K_1, 2*K_1):
            if theta_n[ih] < 0 :
                theta_n[ih] = theta_p[ih]
            
        err = np.linalg.norm(theta_n - theta_p)
        print(theta_n, err)

        if err < 1e-6:
            done = True
        theta_p = np.copy(theta_n)

        S_0 = (1/Z.shape[0])*(xi*Z).T@(xi*Z)
        V =   (1/Z.shape[0])*(symmetric_inversion(G @ W @ (G.T))@ (G @ W @ S_0 @ W @ (G.T)) @ symmetric_inversion(G @ W @ (G.T)))
        print(np.sqrt(V.diagonal()))

        # Define the model
    model = MLIV(J, T, K_1 )

    # Define the optimizer
    optimizer = Adam(model.parameters(), lr=0.001)
    for i in range(10):
        optimizer.zero_grad()
        cost = model(_X = torch.from_numpy(x_1).float(), _Z = torch.from_numpy(Z_raw).float(), _jacobian = torch.from_numpy(jacobian).float(), _Xi = torch.from_numpy(xi).float())
        print(cost.item())
        cost.backward()
        optimizer.step()
        print(cost.item())
    return model


In [9]:
def estimate_blp(seed, T, J, K_1, N, model, use_local=False):
    data_spec = { 'T': T, 'J': J,
                'N': N, 'beta_true': np.array([1,  1]), 'K_1': K_1 }
    data_raw = gen_data(seed, data_spec)
    data = data_transform(data_raw)
    x_1  = data['x_1']
    s    = data['s']
    s_0  = data['s_0']
    v    = data['v']
    X   = data['X']
    Z   = data['Z']
    delta_orig = data['delta']
    Z_list = []
    for col in Z.columns:
        Z_list.append(np.vstack(np.array(Z[col].apply(lambda x: np.array(x)))))
    Z_raw = np.concatenate(Z_list, axis=1)
    if model:
        Z = model.get_mliv(_X = torch.from_numpy(x_1).float(), _Z = torch.from_numpy(Z_raw).float())
        Z = Z.detach().numpy()
    else:
        if use_local:
            Z = local_iv_instruments(Z_raw,  K_1)    
        else:
            Z = blp_instruments(Z_raw,  K_1)    
    W  = symmetric_inversion(Z.T@Z)
    theta_p = np.ones(2*K_1)/2
    done = False
    iterations = 0
    flag = False
    while not done :
        iterations += 1
        if iterations > 20:
            flag = True
            print('flagged')
            return 0,0,True
        theta_1 = theta_p[:K_1]
        theta_2 = theta_p[K_1:]
        delta = compute_delta(theta_2, v, x_1, np.log(s/s_0) , T, J, N, s)
        diff  = 0.0001
        delta_diffs = Parallel(n_jobs=-1)(delayed(compute_diff)(i, theta_2, v, x_1, np.log(s/s_0), T, J, N, s, delta, diff) for i in range(K_1))
        delta_deriv = np.concatenate(delta_diffs, axis=1)    
        xi = delta - (x_1@theta_1).reshape(-1,1)
        g  = np.mean(xi*Z,  axis = 0) 
        jacobian = np.concatenate([-1*x_1,delta_deriv],axis=1)
        moment_grad = np.mean((jacobian[:, :, np.newaxis] * Z[:, np.newaxis, :]), axis = 0)
        G = -1*moment_grad
        #if theta_p[2] > -2:
        #    theta_p[2] = -2.41
        theta_n = theta_p + symmetric_inversion(G @ W @ (G.T)) @ G @ W @ np.reshape(g,[-1,1]).flatten()
        for ih in range(K_1, 2*K_1):
            if theta_n[ih] < 0 :
                theta_n[ih] = theta_p[ih]
            
        err = np.linalg.norm(theta_n - theta_p)
        print(theta_n, err)

        if err < 1e-6:
            done = True
        theta_p = np.copy(theta_n)

        S_0 = (1/Z.shape[0])*(xi*Z).T@(xi*Z)
        V =   (1/Z.shape[0])*(symmetric_inversion(G @ W @ (G.T))@ (G @ W @ S_0 @ W @ (G.T)) @ symmetric_inversion(G @ W @ (G.T)))
        print(np.sqrt(V.diagonal()))
    return (theta_p, np.sqrt(V.diagonal()), flag)


In [10]:
thetas = []
sigmas = []
np.random.seed(12345)
for i in range(20):
    seeds = (np.random.uniform(0,1,2)*100).astype(int)
    model_1 = generate_instruments(seed = seeds[0], T = 50, J = 5, K_1 = 2, N   = 2000)
    if model_1 is None:
        continue
    theta_1, sigma_1, flag_1 = estimate_blp(seed = seeds[1], T = 50, J = 5, K_1 = 2, N   = 2000, model = model_1)
    # logic to avoid convergence issues
    if flag_1:
        continue
    model_2 = generate_instruments(seed = seeds[1], T = 50, J = 5, K_1 = 2, N   = 2000)
    if model_2 is None:
        continue
    theta_2, sigma_2, flag_2 = estimate_blp(seed = seeds[0], T = 50, J = 5, K_1 = 2, N   = 2000, model = model_2)
    # logic to avoid convergence issues
    if flag_2:
        continue
    theta = (theta_1 + theta_2)/2
    sigma = (sigma_1 + sigma_2)/2
    # some logic to avoid the bad estimates
    if np.max(sigma) < 10:
        thetas.append(theta)
        sigmas.append(sigma)

Calculating market shares (250, 1)
Dimension of X vector: 5
Dimension of Z vector: 5
[0.8902671  1.37016482 0.66699284 1.92834558] 1.7255587284164928
[0.39495823 1.04760464 1.19477429 2.41583286]
[1.0087063  1.34061292 0.91682144 1.5517724 ] 0.4681055984180663
[0.61597693 0.59667261 1.32509656 0.87478171]
[0.98654768 1.30165554 0.86606425 1.48181982] 0.09735675104310372
[0.27960772 0.40423232 0.52975131 0.62065394]
[0.98708149 1.30223221 0.86649305 1.48251007] 0.0011304007079396378
[0.25310352 0.36899849 0.48893556 0.57028844]
[0.98708154 1.30223227 0.86649311 1.48251014] 1.200691347866374e-07
[0.25336551 0.36933377 0.48935688 0.5707661 ]
torch.Size([250, 10])
tensor([157.5585, 201.4166, 804.2066, 522.0785], grad_fn=<DiagBackward0>)
1326.28515625
1326.28515625
torch.Size([250, 10])
tensor([0.0071, 0.0131, 0.0369, 0.0312], grad_fn=<DiagBackward0>)
0.06812268495559692
0.06812268495559692
torch.Size([250, 10])
tensor([0.0028, 0.0064, 0.0135, 0.0144], grad_fn=<DiagBackward0>)
0.02784153074

In [10]:
from joblib import Parallel, delayed
import numpy as np

def compute_estimate(i):
    np.random.seed(i + 12345)  # each process gets a different seed
    seeds = (np.random.uniform(0,1,2)*100).astype(int)
    
    model_1 = generate_instruments(seed = seeds[0], T = 50, J = 3, K_1 = 2, N   = 2000)
    if model_1 is None:
        return None, None
    theta_1, sigma_1, flag_1 = estimate_blp(seed = seeds[1], T = 50, J = 3, K_1 = 2, N = 2000, model = model_1)
    if flag_1:
        return None, None
    
    model_2 = generate_instruments(seed = seeds[1], T = 50, J = 3, K_1 = 2, N   = 2000)
    if model_2 is None:
        return None, None
    theta_2, sigma_2, flag_2 = estimate_blp(seed = seeds[0], T = 50, J = 3, K_1 = 2, N = 2000, model = model_2)
    if flag_2:
        return None, None

    theta = (theta_1 + theta_2)/2
    sigma = (sigma_1 + sigma_2)/2
    
    if np.max(sigma) < 20:
        return theta, sigma
    else:
        return None, None

# Use all available CPUs
n_jobs = -1

# Run parallel computation
results = Parallel(n_jobs=n_jobs)(delayed(compute_estimate)(i) for i in range(20))

# Filter out the results that have None values and unzip the results
thetas, sigmas = zip(*[result for result in results if result[0] is not None])

# If you prefer lists instead of tuples:
thetas = list(thetas)
sigmas = list(sigmas)


In [11]:
np.mean(sigmas, axis = 0)

array([0.04011824, 0.04693004, 0.09650867, 0.13570159])

In [12]:
np.mean(thetas-np.array([1,1,1,1]), axis = 0)

array([-0.0062986 ,  0.00025788, -0.00669741, -0.01984472])

In [13]:
# Calculate RMSE for thetas compared to true parameters [1,1,1,1]
rmse = np.sqrt(np.mean((np.array(thetas) - np.array([1,1,1,1]))**2, axis=0))
print("RMSE for each parameter:", rmse)


RMSE for each parameter: [0.02387978 0.03256271 0.06332247 0.09841102]


In [19]:
thetas = []
sigmas = []
np.random.seed(12345)
for i in range(20):
    seeds = (np.random.uniform(0,1,2)*100).astype(int)
    theta, sigma,flag = estimate_blp(seed = seeds[1], T = 100, J = 5, K_1 = 2, N   = 2000, model = None, use_local=True)
    if flag:
        continue
    print(theta, sigma)
    # some logic to avoid the bad estimates
    if np.max(sigma) < 20:
        thetas.append(theta)
        sigmas.append(sigma)

Calculating market shares (500, 1)
Dimension of X vector: 5
Dimension of Z vector: 5
[0.97856818 0.98832956 1.05248911 1.06841878] 1.0468225204684807
[0.0538981  0.05184717 0.09201928 0.08671994]
[0.99718216 1.01594907 1.00310103 1.02335324] 0.07469540440453329
[0.01157256 0.01337427 0.02168173 0.02312197]
[0.99734566 1.01613533 1.00309434 1.02340369] 0.00025301312969950657
[0.0095371  0.01152295 0.01585177 0.01791437]
[0.99734566 1.01613533 1.00309434 1.02340369] 1.6256449712458458e-10
[0.00953881 0.01152303 0.01585508 0.01791369]
[0.99734566 1.01613533 1.00309434 1.02340369] [0.00953881 0.01152303 0.01585508 0.01791369]
Calculating market shares (500, 1)
Dimension of X vector: 5
Dimension of Z vector: 5
[0.97302453 0.96141716 1.02892698 1.03397994] 1.0007778599357733
[0.06130102 0.05775265 0.09998076 0.09311139]
[0.99358254 0.98358569 0.99096518 1.00058088] 0.05891239785164028
[0.01491627 0.01361329 0.02556717 0.02271086]
[0.99369876 0.98368229 0.9909891  1.00061743] 0.00015730822586

In [20]:
np.mean(sigmas, axis = 0)

array([0.01100356, 0.01130337, 0.01663625, 0.01671496])

In [21]:
bias = np.mean(thetas-np.array([1,1,1,1]), axis = 0)
print("Bias for each parameter:")
print(f"Parameter 1: {bias[0]:.6f}")
print(f"Parameter 2: {bias[1]:.6f}") 
print(f"Parameter 3: {bias[2]:.6f}")
print(f"Parameter 4: {bias[3]:.6f}")

Bias for each parameter:
Parameter 1: 0.004633
Parameter 2: 0.004657
Parameter 3: 0.002550
Parameter 4: 0.006179


In [22]:
# Calculate RMSE for thetas compared to true parameters [1,1,1,1]
rmse = np.sqrt(np.mean((np.array(thetas) - np.array([1,1,1,1]))**2, axis=0))
print("RMSE for each parameter:", rmse)


RMSE for each parameter: [0.01079733 0.0115148  0.01286018 0.01512814]


In [14]:
from joblib import Parallel, delayed
import numpy as np

def compute_estimate(i):
    np.random.seed(i + 12345)  # each process gets a different seed
    seeds = (np.random.uniform(0,1,2)*100).astype(int)
    theta, sigma, flag = estimate_blp(seed=seeds[1], T=100, J=3, K_1=2, N=2000, model=None)
    if flag or np.max(sigma) >= 20:
        return None, None
    else:
        return theta, sigma

# Use all available CPUs
n_jobs = 8

# Run parallel computation
results = Parallel(n_jobs=n_jobs, verbose=100)(delayed(compute_estimate)(i) for i in range(20))

# Filter out the results that have None values and unzip the results
thetas, sigmas = zip(*[result for result in results if result[0] is not None])

# If you prefer lists instead of tuples:
thetas = list(thetas)
sigmas = list(sigmas)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:  2.8min
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:  3.4min
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:  4.0min
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:  4.1min
[Parallel(n_jobs=8)]: Done   6 out of  20 | elapsed:  5.5min remaining: 12.8min
[Parallel(n_jobs=8)]: Done   7 out of  20 | elapsed:  5.8min remaining: 10.7min
[Parallel(n_jobs=8)]: Done   8 out of  20 | elapsed:  6.4min remaining:  9.5min
[Parallel(n_jobs=8)]: Done   9 out of  20 | elapsed:  7.5min remaining:  9.2min
[Parallel(n_jobs=8)]: Done  10 out of  20 | elapsed:  9.0min remaining:  9.0min
[Parallel(n_jobs=8)]: Done  11 out of  20 | elapsed:  9.2min remaining:  7.5min
[Parallel(n_jobs=8)]: Done  12 out of  20 | elapsed:  9.7min remaining:  6.5min
[Parallel(n_jobs=8)]: Done  13 out of  20 | elapsed:  9.8min

In [17]:
np.mean(sigmas, axis = 0)

array([0.30213628, 0.30644669, 0.74783923, 0.92560819])

In [18]:
# Calculate bias compared to true parameters [1,1,1,1]
bias = np.mean(np.array(thetas) - np.array([1,1,1,1]), axis=0)
print("\nBias for each parameter:")
print(f"Parameter 1: {bias[0]:.6f}")
print(f"Parameter 2: {bias[1]:.6f}")
print(f"Parameter 3: {bias[2]:.6f}") 
print(f"Parameter 4: {bias[3]:.6f}")

# Calculate RMSE
rmse = np.sqrt(np.mean((np.array(thetas) - np.array([1,1,1,1]))**2, axis=0))
print("\nRMSE for each parameter:")
print(f"Parameter 1: {rmse[0]:.6f}")
print(f"Parameter 2: {rmse[1]:.6f}")
print(f"Parameter 3: {rmse[2]:.6f}")
print(f"Parameter 4: {rmse[3]:.6f}")



Bias for each parameter:
Parameter 1: 0.016862
Parameter 2: -0.045092
Parameter 3: -0.006751
Parameter 4: -0.062293

RMSE for each parameter:
Parameter 1: 0.175088
Parameter 2: 0.189007
Parameter 3: 0.403076
Parameter 4: 0.331963
